In [47]:
# First let's do an import. If you get an Import Error, double check that your Kernel is correct..

from dotenv import load_dotenv


In [48]:
# Next it's time to load the API keys into environment variables
# If this returns false, see the next cell!

load_dotenv(override=True)

True

In [49]:
# Check the key - if you're not using OpenAI, check whichever key you're using! Ollama doesn't need a key.

import os
openai_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")
    


OpenAI API Key exists and begins AIzaSyD2


In [57]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

# 1) Trust corporate certs (must be the provided amdcerts.pem bundle)
os.environ["REQUESTS_CA_BUNDLE"] = r"C:\amdcerts.pem"
os.environ["SSL_CERT_FILE"]      = os.environ["REQUESTS_CA_BUNDLE"]

# 2) Bypass proxy for autox
os.environ["NO_PROXY"] = ",".join(filter(None, [
    os.getenv("NO_PROXY",""),
    ".autox.corp.amdocs.azr",
    "chat.autox.corp.amdocs.azr",
    "localhost","127.0.0.1"
]))
os.environ["no_proxy"] = os.environ["NO_PROXY"]

# 3) Sanity check the Proxy host (not agent-client)
import requests
print(requests.get(
    "https://chat.autox.corp.amdocs.azr/api/v1/platforms/list",
    headers={"accept":"application/json"},
    proxies={"http": None, "https": None},
    timeout=30
).status_code)

200


In [ ]:

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage,AIMessage
import os

AUTOX_API_KEY  = os.getenv("AUTOX_API_KEY")
NTNET_USERNAME = (os.getenv("NTNET_USERNAME") or "").strip()

llm = AzureChatOpenAI(
    azure_endpoint="https://chat.autox.corp.amdocs.azr/api/v1/proxy",
    api_key=AUTOX_API_KEY,
    azure_deployment="gpt-4o-128k",
    model="gpt-4o-128k",
    temperature=0.1,
    openai_api_version="2023-05-15",
    default_headers={"username": NTNET_USERNAME, "application": "testing-proxyapi"},
)

In [59]:
resp = llm.invoke([HumanMessage(content="Hello, who are you?")])
print(resp.content)

Hello! I'm an AI assistant created to help with information, answer questions, and provide support on a variety of topics. Think of me as a friendly and knowledgeable companion here to assist you. How can I help you today? 😊


In [62]:
# And now call it! Any problems, head to the troubleshooting guide
# This uses GPT 4.1 nano, the incredibly cheap model
# The APIs guide (guide 9) has exact instructions for using even cheaper or free alternatives to OpenAI
# If you get a NameError, head to the guides folder (guide 6) to learn about NameErrors - always instantly fixable
from langchain_core.messages import HumanMessage,AIMessage
history: list = []
def ask(user_text):
    msgs = history + [HumanMessage(content=user_text)]
    ai=llm.invoke(msgs)
    history.extend([HumanMessage(content=user_text),AIMessage(content=ai.content)])
    return ai.content

print(ask("Hi, my name is Nikhil. Acknowledge."))
print(ask("What is my name?"))
print(ask("Summarize our conversation in one sentence."))


Hello, Nikhil! Nice to meet you. How can I assist you today?
Your name is Nikhil! 😊 How can I help you today?
You introduced yourself as Nikhil, and I acknowledged your name while offering assistance.


In [ ]:
# And now - let's ask for a question:

question = "Please propose a hard, challenging question to assess someone's IQ. Respond only with the question."
messages = [{"role": "user", "content": question}]
